In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

### Import First Name Files

In [5]:
#path = r'\\CPFSPRDAPAWSE01\userdata$\shilts\Documents\Projects\name-anagram\names'
path = r'C:\Users\stefa\GitHub\anagrammer-master\anagrammer-master\names'
last_path  = r'C:\Users\stefa\GitHub\anagrammer-master\anagrammer-master\Names_2010Census.csv'
# date = 2000
# df = pd.read_csv(path+'\yob'+str(date)+'.txt',names=['name','gender','count'])
# df['year'] = date

In [6]:
date = 1880
df = pd.read_csv(path+'\yob'+str(1880)+'.txt',names=['name','gender','count'])
df['year'] = date

for date in range(1880,2019):
    df_temp = pd.read_csv(path+'\yob'+str(date)+'.txt',names=['name','gender','count'])
    df_temp['year'] = date
    df = df.append(df_temp)
df_firstname = pd.DataFrame(df.groupby(['name','gender'])['count'].sum())    

In [7]:
df_firstname.reset_index(inplace=True)
df_firstname.columns = ['name', 'gender','count']


### Run First-Name Finder

In [9]:
alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']


In [10]:
def letter_count(name):
    letter_list = []
    name = name.upper()
    for letter in alphabet:
        letter_list.append(name.count(letter))
    return np.array(letter_list)

def name_compare(name1,name2):
    letter_array = letter_count(name1) - letter_count(name2)
    if min(letter_array)<0:
        str_letter = ''
    else:
        str_letter = ''
        for x in range(26):
            str_letter = str_letter+letter_array[x]*alphabet[x]
    vowel_count = letter_array[0]+letter_array[4]+letter_array[8]+letter_array[14]+letter_array[20]+letter_array[24]
    return str_letter,vowel_count

In [11]:
def first_name_list(testname, df, min_length, max_length, vowels):
    df['eligible'] = df['name'].apply(lambda x: name_compare(testname,x)[0])
    df['vowels'] = df['name'].apply(lambda x: name_compare(testname,x)[1])
    df['rem_count'] = df['eligible'].apply(lambda x: len(x))
    return df[(df['eligible']!='')&(df['vowels']>=vowels)&(df['rem_count']>min_length)&(df['rem_count']<max_length)]


### Last Name Pairs Parser
- Calculate the percentages of consecutive letter pairs based on historic Census data
- Use percentages to create a minimized error

In [12]:
df_lastname = pd.read_csv(last_path)#,names=['name','gender','count'])
df_lastname = df_lastname.iloc[:-1]
df_lastname.sort_values(by='count',ascending=False)

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
1,JOHNSON,2,1932812,655.24,1483.42,58.97,34.63,0.54,0.94,2.56,2.36
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49
3,BROWN,4,1437026,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
4,JONES,5,1425470,483.24,3004.80,55.19,38.48,0.44,1,2.61,2.29
...,...,...,...,...,...,...,...,...,...,...,...
161398,STENGELE,160975,100,0.03,90034.08,98,(S),(S),0,0,0
161397,STRUECKER,160975,100,0.03,90034.04,99,0,0,(S),0,(S)
161396,STRULOVITCH,160975,100,0.03,90034.01,97,(S),0,0,0,(S)
161395,STEUP,160975,100,0.03,90033.98,97,(S),0,0,0,(S)


In [35]:
# pair_probs = np.array(np.zeros(676))
# for testname in df_lastname['name'].values:
#     for x in range(26):
#         for y in range(26):
#             str_chk = alphabet[x]+alphabet[y]
#             try:
#                 pair_probs[x*26+y] += testname.count(str_chk)
#             except:
#                 pass
#                 #print(testname)
# pair_probs

In [11]:
trio_probs = []
trio_letters = []

for x in range(26):
    for y in range(26):
        for z in range(26):
            str_chk = alphabet[x]+alphabet[y]+alphabet[z]
            triplet = 0
            for testname in df_lastname['name'].values:
                try:
                    triplet += testname.count(str_chk)
                except:
                    pass
            trio_probs.append(triplet)
            trio_letters.append(str_chk)
            
            


In [12]:
df_probs = pd.DataFrame(data = [trio_letters,list(trio_probs)]).T
df_probs.columns = ['pairs','counts']
df_probs['probs'] = df_probs['counts']/df_probs['counts'].sum()
df_probs.head(5)

,pairs,counts,probs
0,AAA,1,1.24457e-06
1,AAB,23,2.86252e-05
2,AAC,26,3.23589e-05
3,AAD,27,3.36035e-05
4,AAE,4,4.97829e-06


In [13]:
df_probs.sort_values(by='probs',ascending=False).head()

SyntaxError: invalid syntax (<ipython-input-13-d3bb20310fc7>, line 1)

In [34]:
df_probs.to_csv(r'C:\Users\stefa\GitHub\anagrammer-master\anagrammer-master\trio_probs.csv')
df_firstname.to_csv(r'C:\Users\stefa\GitHub\anagrammer-master\anagrammer-master\df_firstname.csv')